# tem Beneficios a entrega contínua?

## Revisar literatura para encontrar melhores perguntas de pesquisa

## Analisar dataset para estabelecer quais projetos tem entrega contínua



In [1]:
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId
from datetime import timedelta
import numpy as np
import datetime
import re
client = MongoClient()
db = client.smartshark_2_2

### 1) Tem integração contínua - OK

In [2]:
vcs_systems = pd.DataFrame(list(db.vcs_system.find()))
travisBuilds = pd.DataFrame(list(db.travis_build.find()))
projetosComBuilds = travisBuilds['vcs_system_id'].unique()
print(vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna().count())
vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()

_id                46
url                46
project_id         46
last_updated       46
repository_type    46
repository_file    46
dtype: int64


,_id,url,project_id,last_updated,repository_type,repository_file
0,5bef24dc35e3ea2b7bc08208,https://github.com/apache/commons-rdf.git,5beebd870da3ef21a62b756e,2018-11-16 21:13:16.874,git,5dcbf72e2ecf182bf5d3ab89
2,58bfd79dfa94f2c7f0d32551,https://github.com/apache/commons-math,58bfd5860da3ef7cef464c78,2018-02-15 11:35:53.876,git,5dcbf5ce2ecf182bf5d37773
3,5b1fa04730a71b06bc3734c1,https://github.com/apache/mahout,5b1f9fbb0da3ef49b7d0a480,2018-06-12 12:28:23.953,git,5dcbf61c2ecf182bf5d382c4
4,5b27c64830a71b06bceb0ead,https://github.com/apache/opennlp.git,5b27c4cc0da3ef789daec0a1,2018-06-18 16:48:40.415,git,5dcbf6262ecf182bf5d3852f
5,5b34a9c830a71b06bc3d9ef3,https://github.com/apache/calcite.git,5b34a9210da3ef789cc240ec,2018-06-28 11:26:32.261,git,5dcbf6362ecf182bf5d38708
6,5b3b305530a71b06bc5bba5b,https://github.com/apache/cayenne.git,5b3b2d520da3ef789ec953ea,2018-07-03 10:14:13.878,git,5dcbf63c2ecf182bf5d387d8
8,5da726db3dfe5ddb85120d96,https://github.com/apache/bigtop.git,5da724f20da3ef03c84bcf48,2019-10-16 16:19:07.019,git,5dcbf7d72ecf182bf5d3c55d
9,5c87a755504acf99a4dbf833,https://github.com/apache/commons-bcel.git,5c87a7300da3ef6c8a4bf6d8,2019-03-12 13:34:29.466,git,5dcbf7602ecf182bf5d3b19e
10,5bed279735e3ea2b7b9d059e,https://github.com/apache/commons-beanutils.git,5bed276c0da3ef21a58b337a,2018-11-15 09:00:23.400,git,5dcbf7172ecf182bf5d3a90b
11,5bed5a4635e3ea2b7ba10f97,https://github.com/apache/commons-codec.git,5bed58010da3ef21a58b337b,2018-11-15 12:36:38.541,git,5dcbf7182ecf182bf5d3a91d


### 2) Pegar as tags do projetos de integração contínua? - OK

In [3]:
tags = pd.DataFrame(list(db.tag.find()))
idsVcsComBuilds = vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()._id

In [4]:
tagsDosProjComBuild = tags.query('vcs_system_id.isin(@idsVcsComBuilds)').drop_duplicates().dropna()

In [5]:
tagsDosProjComBuild

,_id,name,commit_id,vcs_system_id,message,tagger_id,date,date_offset
3,5bef24e783ad2f56e01ca88c,0.3.0-incubating,5bef24e783ad2f56e01ca88a,5bef24dc35e3ea2b7bc08208,Apache Commons RDF 0.3.0-incubating\n\nhttps:/...,58c1133102ca40f8bfb1e2a9,2016-11-15 13:28:17,0.0
5,5bef254783ad2f56d01caefd,0.1.0-incubating,5bef254683ad2f56d01caefb,5bef24dc35e3ea2b7bc08208,[maven-release-plugin] copy for tag 0.1.0-incu...,58bfd1cb02ca40f8bf1485a4,2015-05-07 14:10:41,120.0
27,58bfd7a53cff546cd3b42b9e,MATH_3_4_1_RC1,58bfd7a53cff546cd3b42b9a,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4.1 RC1 tag.\n...,58bfd79d3cff546cc7b42a28,2015-01-08 16:51:54,60.0
28,58bfd7a53cff546cd8b42c1b,MATH_3_4_RC2,58bfd7a53cff546cd8b42c17,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4 RC2 tag.\n--...,58bfd79d3cff546cc7b42a28,2014-12-19 14:41:18,60.0
29,58bfd7b83cff546cc8b43dab,MATH_1_2,58bfd7b83cff546cc8b43da7,58bfd79dfa94f2c7f0d32551,MATH_1_2\n,58bfd7a53cff546ccbb42b96,2008-02-24 05:56:08,0.0
...,...,...,...,...,...,...,...,...
4780,615e91666511689f63ec2c87,qpid-jms-0.1.0-beta,615e91666511689f63ec2c85,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag qpid-jms-0...,5a031143efa7b46cc724c03b,2015-03-09 18:24:25,0.0
4781,615e91ae6511689f63ec816c,1.2.0,615e91ae6511689f63ec816a,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.2.0\n,5e71cec61c4f2a5a2b6418d7,2021-09-24 12:36:32,60.0
4782,615e91af6511689f63ec8221,1.1.0,615e91af6511689f63ec821f,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.1.0\n,5f27cd89af02e2d6de7c7cdc,2021-07-05 15:35:44,60.0
4783,615e91b06511689f63ec8294,1.0.0,615e91b06511689f63ec8292,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.0.0\n,5f27cd89af02e2d6de7c7cdc,2021-05-19 14:58:57,60.0


### 3) Determinei oq vou considerar Tag de Release - OK
##### Tags de release não tem no nome os termos 'RC', 'candidate', 'test', 'beta', 'alfa', 'unreleased', snapshot, siteupdate, freeze, sandbox, migration

In [6]:
tagsDeRelase = tagsDosProjComBuild[
    tagsDosProjComBuild['name']
    .str.contains('rc|RC|candidate|CANDIDATE|test|TEST|beta|BETA|alpha|ALPHA|unreleased|UNRELEASED|snapshot|SNAPSHOT|siteupdate|SITEUPDATE|freeze|FREEZE|sandbox|SANDBOX|migration|MIGRATION|dev|DEV')==False].drop_duplicates().dropna()

In [7]:
tagsDeRelase.name.to_csv('tags.csv', index=False)

In [8]:
tagsDeRelase

,_id,name,commit_id,vcs_system_id,message,tagger_id,date,date_offset
3,5bef24e783ad2f56e01ca88c,0.3.0-incubating,5bef24e783ad2f56e01ca88a,5bef24dc35e3ea2b7bc08208,Apache Commons RDF 0.3.0-incubating\n\nhttps:/...,58c1133102ca40f8bfb1e2a9,2016-11-15 13:28:17,0.0
5,5bef254783ad2f56d01caefd,0.1.0-incubating,5bef254683ad2f56d01caefb,5bef24dc35e3ea2b7bc08208,[maven-release-plugin] copy for tag 0.1.0-incu...,58bfd1cb02ca40f8bf1485a4,2015-05-07 14:10:41,120.0
29,58bfd7b83cff546cc8b43dab,MATH_1_2,58bfd7b83cff546cc8b43da7,58bfd79dfa94f2c7f0d32551,MATH_1_2\n,58bfd7a53cff546ccbb42b96,2008-02-24 05:56:08,0.0
32,58bfd7c33cff546cd1b447ff,MATH_3_1,58bfd7c33cff546cd1b447fb,58bfd79dfa94f2c7f0d32551,MATH_3_1\n,58bfd7a53cff546ccab42b89,2012-12-23 18:09:22,0.0
33,58bfd7cd3cff546cd5b44d22,MATH_3_5,58bfd7cd3cff546cd5b44d1c,58bfd79dfa94f2c7f0d32551,RC2 becomes the 3.5 official version.\n-----BE...,58bfd79d3cff546cc7b42a28,2015-04-17 14:39:07,120.0
...,...,...,...,...,...,...,...,...
4779,615e91646511689f63ec2b38,qpid-jms-0.1.0,615e91646511689f63ec2b36,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag qpid-jms-0...,5a031143efa7b46cc724c03b,2015-03-16 18:45:53,0.0
4781,615e91ae6511689f63ec816c,1.2.0,615e91ae6511689f63ec816a,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.2.0\n,5e71cec61c4f2a5a2b6418d7,2021-09-24 12:36:32,60.0
4782,615e91af6511689f63ec8221,1.1.0,615e91af6511689f63ec821f,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.1.0\n,5f27cd89af02e2d6de7c7cdc,2021-07-05 15:35:44,60.0
4783,615e91b06511689f63ec8294,1.0.0,615e91b06511689f63ec8292,615e9105f73e2aa390c8a69b,[maven-release-plugin] copy for tag 1.0.0\n,5f27cd89af02e2d6de7c7cdc,2021-05-19 14:58:57,60.0


### 4) Filtrar builds que tem commits de tag de release - OK

In [9]:
commitsDeRelease = tagsDeRelase.commit_id.unique()
buildsIntegrados = travisBuilds.query('commit_id.isin(@commitsDeRelease)')
stateIndesejados = ['failed', 'canceled']
BuildsValidos = buildsIntegrados.query('~state.isin(@stateIndesejados)')

In [10]:
BuildsValidos.state.value_counts()

passed     216
errored     48
Name: state, dtype: int64

### 5) Analisar quais projetos tem builds de entrega ok 

In [11]:
vcsComEntrega = BuildsValidos.vcs_system_id.unique()
print(vcs_systems.query('_id.isin(@vcsComEntrega)').drop_duplicates().dropna().count())
vcs_systems.query('_id.isin(@vcsComEntrega)').drop_duplicates().dropna()

_id                34
url                34
project_id         34
last_updated       34
repository_type    34
repository_file    34
dtype: int64


,_id,url,project_id,last_updated,repository_type,repository_file
0,5bef24dc35e3ea2b7bc08208,https://github.com/apache/commons-rdf.git,5beebd870da3ef21a62b756e,2018-11-16 21:13:16.874,git,5dcbf72e2ecf182bf5d3ab89
4,5b27c64830a71b06bceb0ead,https://github.com/apache/opennlp.git,5b27c4cc0da3ef789daec0a1,2018-06-18 16:48:40.415,git,5dcbf6262ecf182bf5d3852f
5,5b34a9c830a71b06bc3d9ef3,https://github.com/apache/calcite.git,5b34a9210da3ef789cc240ec,2018-06-28 11:26:32.261,git,5dcbf6362ecf182bf5d38708
6,5b3b305530a71b06bc5bba5b,https://github.com/apache/cayenne.git,5b3b2d520da3ef789ec953ea,2018-07-03 10:14:13.878,git,5dcbf63c2ecf182bf5d387d8
8,5da726db3dfe5ddb85120d96,https://github.com/apache/bigtop.git,5da724f20da3ef03c84bcf48,2019-10-16 16:19:07.019,git,5dcbf7d72ecf182bf5d3c55d
9,5c87a755504acf99a4dbf833,https://github.com/apache/commons-bcel.git,5c87a7300da3ef6c8a4bf6d8,2019-03-12 13:34:29.466,git,5dcbf7602ecf182bf5d3b19e
11,5bed5a4635e3ea2b7ba10f97,https://github.com/apache/commons-codec.git,5bed58010da3ef21a58b337b,2018-11-15 12:36:38.541,git,5dcbf7182ecf182bf5d3a91d
12,5bed25af35e3ea2b7b8bb711,https://github.com/apache/commons-collections.git,5bed254e0da3ef21a62b756d,2018-11-15 08:52:15.394,git,5dcbf7162ecf182bf5d3a8bb
13,5bed61cc35e3ea2b7b2a1805,https://github.com/apache/commons-compress.git,5bed583a0da3ef21a58b337c,2018-11-15 13:08:44.599,git,5dcbf71b2ecf182bf5d3a931
14,5bed83ca35e3ea2b7be8d7c9,https://github.com/apache/commons-configuratio...,5bed83440da3ef21a2107882,2018-11-15 15:33:46.623,git,5dcbf71d2ecf182bf5d3a999


### 6) Desses projetos quais tem uma quantidade relevante de entregas que permita a nossa análise (>3 Builds com integração) - OK


In [12]:
BuildsValidosPorCommit = BuildsValidos[['_id', 'vcs_system_id', 'commit_id', 'number', 'state', 'started_at', 'finished_at']].groupby(['commit_id'], as_index=False, sort=False).max()

In [13]:
vcsComCD = BuildsValidosPorCommit[['vcs_system_id','_id']].groupby(['vcs_system_id'], as_index=False).agg('count').sort_values(by=['_id'], ascending=False)
vcsComCD = vcsComCD.query('_id > 2')
print(vcsComCD.count())
vcsComCD

vcs_system_id    21
_id              21
dtype: int64


,vcs_system_id,_id
11,5bd0268235e3ea2b7bbfdbae,17
4,5b43737430a71b06bcbdd168,13
0,5b23df3c30a71b06bc01e16a,12
7,5b7ffe9530a71b06bc70c038,9
1,5b27c64830a71b06bceb0ead,7
15,5bed61cc35e3ea2b7b2a1805,6
25,5d8ccfb33dfe5ddb8590d841,6
21,5c3f0a7c621a9a77b35f3add,6
30,608b9b33c26a57681ebf90b6,5
24,5cd2806d504acf99a4cd728c,5


## Validação de continuidade de uso de CD (em paralelo uma validação manual)

Quantos Releases tem a partir da adoção  X Quantos Builds com commits de Release a partir da data da adoção de CD 

# Fim caracterização

### 7) Percorrer os projetos e coletar seus dados

In [14]:
vcsList= []
for index, row in vcsComCD.iterrows():
    vcsList.append(row.vcs_system_id)
vcsList

[ObjectId('5bd0268235e3ea2b7bbfdbae'),
 ObjectId('5b43737430a71b06bcbdd168'),
 ObjectId('5b23df3c30a71b06bc01e16a'),
 ObjectId('5b7ffe9530a71b06bc70c038'),
 ObjectId('5b27c64830a71b06bceb0ead'),
 ObjectId('5bed61cc35e3ea2b7b2a1805'),
 ObjectId('5d8ccfb33dfe5ddb8590d841'),
 ObjectId('5c3f0a7c621a9a77b35f3add'),
 ObjectId('608b9b33c26a57681ebf90b6'),
 ObjectId('5cd2806d504acf99a4cd728c'),
 ObjectId('5bbe0f4057674ee16793f245'),
 ObjectId('5c87a96c504acf99a4de5a18'),
 ObjectId('5d91bc213dfe5ddb85cc1e0e'),
 ObjectId('5bed83ca35e3ea2b7be8d7c9'),
 ObjectId('615e9105f73e2aa390c8a69b'),
 ObjectId('5b3b305530a71b06bc5bba5b'),
 ObjectId('5b76bead30a71b06bcadc156'),
 ObjectId('5c87a755504acf99a4dbf833'),
 ObjectId('5b47693d30a71b06bc04ba2e'),
 ObjectId('5bc5d34957674ee16701b68a'),
 ObjectId('5bee7d5b35e3ea2b7ba03bfa')]

Montar a primeira tabela, que para cada vcs_system_id, busca project_id, issues_system_id, data da adoção de CD, primeira build de CD, quantas builds antes e depois do CD, campo de análise manual se tem a CD suspeita

In [15]:
issue_system = pd.DataFrame(list(db.issue_system.find()))
projects = pd.DataFrame(list(db.project.find()))
listaDosProjetosComCD = [] 
issues_system_ids = []
for vcs in vcsList:
    # Buscar informações sobre o projeto
    project_id = vcs_systems.query('_id == @vcs').iloc[0,2] 

    #nome do projeto
    name = projects.query('_id == @project_id').iloc[0,1]     

    # Informações sobre o issue system
    issues_system_id = issue_system.query('project_id == @project_id').iloc[0,0]
    issues_system_ids.append(issues_system_id)   

    # Pegar o commit da build considerada como a build de adoção (desconsiderando o primeiro)
    commit_id_DaBuilld = BuildsValidosPorCommit.query('vcs_system_id == @vcs').sort_values(by=['started_at']).iloc[1,0]
    

    # Pegar a release com o commit de build de adoção
    id_tag_Adocao_CD = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,0]
    nome_da_releasa = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,1]
    data_de_adocao = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,6]

    listaDosProjetosComCD.append([name , vcs,project_id, issues_system_id, commit_id_DaBuilld, id_tag_Adocao_CD, nome_da_releasa, data_de_adocao  ])

dfProjetosComCD = pd.DataFrame(listaDosProjetosComCD, columns=['name', 'vcs_system_id', 'project_id', 'issues_system_id', 'commit_id', 'tag_id','adoption_release', 'cd_adoption_date' ])
dfProjetosComCD

,name,vcs_system_id,project_id,issues_system_id,commit_id,tag_id,adoption_release,cd_adoption_date
0,nifi,5bd0268235e3ea2b7bbfdbae,5bd022e30da3ef1a10738359,5bdafdb2f16c872096848fba,5bd026e5bad86c1de7dd18ef,5bd026e5bad86c1de7dd18f1,nifi-1.0.0,2016-08-31 05:45:12
1,kylin,5b43737430a71b06bcbdd168,5b43728b0da3ef07106a790f,5b45f47356677a15366cae54,5b437ae46d2aba30544aa805,5b437ae56d2aba30544aa807,kylin-1.5.4.1-HBase1.x,2016-09-28 10:06:24
2,struts,5b23df3c30a71b06bc01e16a,5b2225520da3ef3e52ece5e2,5b3514966e373d64706ea578,5b23df79a1501a253cd0d5fb,5b23df79a1501a253cd0d5fd,STRUTS_2_5_4,2016-10-06 19:04:10
3,storm,5b7ffe9530a71b06bc70c038,5b7ffd4e0da3ef049126bdbd,5b868868e68f507fbce49133,5b7fffad58a6d70319e31dbf,5b7fffad58a6d70319e31dc1,v1.1.0,2017-03-21 16:56:03
4,opennlp,5b27c64830a71b06bceb0ead,5b27c4cc0da3ef789daec0a1,5b3b1dcd428fc37a4b3c804e,5b27c64eeee553715e77d1f9,5b27c64eeee553715e77d1fb,opennlp-1.7.2,2017-02-01 16:03:40
5,commons-compress,5bed61cc35e3ea2b7b2a1805,5bed583a0da3ef21a58b337c,5bed83e88fc5e9768f66c998,5bed61df65108826c67b0fd9,5bed61df65108826c67b0fdb,1.14,2017-05-14 20:26:51
6,httpcomponents-core,5d8ccfb33dfe5ddb8590d841,5d8cb4a10da3ef0b68077801,5d8dbf905227b202863a5e83,5d8cd1b653cc927575ce3dd2,5d8cd1b653cc927575ce3dd6,4.4.11,2019-01-18 11:09:54
7,manifoldcf,5c3f0a7c621a9a77b35f3add,5c3f09db0da3ef512137483f,5c57ee84cf244d7545c7f950,5c3f2e2566ca241a78525008,5c3f2e2566ca241a7852500a,release-2.7.1,2017-05-10 15:50:59
8,curator,608b9b33c26a57681ebf90b6,608b9b0227ad48a9ceb48654,60fac2459b37c1166a9cbc13,608b9b38e61739c47b70f003,608b9b38e61739c47b70f005,apache-curator-4.2.0,2019-03-03 19:54:29
9,httpcomponents-client,5cd2806d504acf99a4cd728c,5cd27fe20da3ef671cda02e7,5d7778c99f03373d37512b7e,5cd284514eb92640bb52e439,5cd284514eb92640bb52e43b,4.5.4,2017-11-30 10:18:48


### 8) Buscar as issues dos projetos analisados para contabilizar as metricas

In [16]:
issues = pd.DataFrame(list(db.issue.find()))
issuesProjetosCD = issues.query('issue_system_id.isin(@issues_system_ids)')

### 9) Contabilizar métricas


#### 1) Time to market (tempo entre releases)
Tempo entre releases (tempo medio)


Algoritmo Final:

In [17]:
difference_per_project = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    tmp_date = dfProjetosComCD.iloc[i,7]
    ttm_tags_no_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date < @tmp_date ').sort_values(by='date')
    ttm_tags_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date >= @tmp_date ').sort_values(by='date')
    average_no_cd = (ttm_tags_no_cd.iloc[ttm_tags_no_cd.shape[0] - 1,6] - ttm_tags_no_cd.iloc[0,6])/(ttm_tags_no_cd.shape[0] - 1)
    average_cd = (ttm_tags_cd.iloc[ttm_tags_cd.shape[0] - 1,6] - ttm_tags_cd.iloc[0,6])/(ttm_tags_cd.shape[0] - 1)
    difference_per_project.append((tmp_name, tmp_vcs, average_no_cd - average_cd))


In [18]:
df_tmt = pd.DataFrame(difference_per_project, columns=['name', 'vcs_system_id', 'noCD_CD'])
df_tmt

,name,vcs_system_id,noCD_CD
0,nifi,5bd0268235e3ea2b7bbfdbae,-11 days +23:51:16.033333334
1,kylin,5b43737430a71b06bcbdd168,-27 days +13:08:18.656126482
2,struts,5b23df3c30a71b06bc01e16a,12 days 11:58:57.881668947
3,storm,5b7ffe9530a71b06bc70c038,35 days 21:38:48.700000
4,opennlp,5b27c64830a71b06bceb0ead,321 days 14:23:20.900000
5,commons-compress,5bed61cc35e3ea2b7b2a1805,11 days 12:08:28.896296296
6,httpcomponents-core,5d8ccfb33dfe5ddb8590d841,-97 days +06:29:31.923076924
7,manifoldcf,5c3f0a7c621a9a77b35f3add,-29 days +05:32:47.120535714
8,curator,608b9b33c26a57681ebf90b6,-116 days +23:03:21.160919541
9,httpcomponents-client,5cd2806d504acf99a4cd728c,-21 days +20:32:47.027777778


In [19]:
temp_delta_zero = datetime.datetime.now() - datetime.datetime.now()
print('Projetos que diminuiram seu Time to Market: ', df_tmt.query('noCD_CD > @temp_delta_zero').count().iloc[0])
print('Projetos que aumentaram seu Time to Market: ', df_tmt.query('noCD_CD < @temp_delta_zero').count().iloc[0]) 


Projetos que diminuiram seu Time to Market:  10
Projetos que aumentaram seu Time to Market:  11


In [20]:
difference_per_project = []
list_builds = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    tmp_adoption_date = dfProjetosComCD.iloc[i,7]
    tmp_df = tagsDeRelase.query('vcs_system_id == @tmp_vcs').sort_values(by='date')
    for j in range(tmp_df.shape[0]):
        tmp_release_date = tmp_df.iloc[j,6]
        if j != 0:
            tmp_date_diference = tmp_release_date - tmp_df.iloc[j-1,6]
            cd_flag = tmp_release_date >= tmp_adoption_date
            release_name = tmp_df.iloc[j,1]
            list_builds.append([tmp_name,tmp_vcs, release_name, tmp_release_date, tmp_date_diference, cd_flag ])                    
df_tmt_builds = pd.DataFrame(list_builds, columns=['name','vcs_system_id', 'release_name','release_date', 'deltatime_last_release', 'isCD'])    

In [21]:
mean_release_tax_no_cd = []
mean_release_tax_cd = []
median_release_tax_no_cd = []
median_release_tax_cd = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    mean_release_tax_no_cd.append(df_tmt_builds.query('vcs_system_id == @tmp_vcs and isCD == False').deltatime_last_release.mean())
    mean_release_tax_cd.append(df_tmt_builds.query('vcs_system_id == @tmp_vcs and isCD == True').deltatime_last_release.mean())
    median_release_tax_no_cd.append(df_tmt_builds.query('vcs_system_id == @tmp_vcs and isCD == False').deltatime_last_release.median())
    median_release_tax_cd.append(df_tmt_builds.query('vcs_system_id == @tmp_vcs and isCD == True').deltatime_last_release.median())
dfProjetosComCD['mean_release_tax_no_cd'] = mean_release_tax_no_cd
dfProjetosComCD['mean_release_tax_cd'] = mean_release_tax_cd
dfProjetosComCD['median_release_tax_no_cd'] = median_release_tax_no_cd
dfProjetosComCD['median_release_tax_cd'] = median_release_tax_cd

In [22]:
print('Projetos com intervalo media entre releases maior após CD:', dfProjetosComCD.query('mean_release_tax_no_cd < mean_release_tax_cd').count().iloc[0] )
print('Projetos com intervalo media entre releases menor após CD:', dfProjetosComCD.query('mean_release_tax_no_cd >= mean_release_tax_cd').count().iloc[0] )
print('Projetos com mediana de intervalo entre releases maior após CD:', dfProjetosComCD.query('median_release_tax_no_cd < median_release_tax_cd').count().iloc[0] )
print('Projetos com mediana de intervalo entre releases menor após CD:', dfProjetosComCD.query('median_release_tax_no_cd >= median_release_tax_cd').count().iloc[0] )

Projetos com intervalo media entre releases maior após CD: 11
Projetos com intervalo media entre releases menor após CD: 10
Projetos com mediana de intervalo entre releases maior após CD: 15
Projetos com mediana de intervalo entre releases menor após CD: 6


#### 2) Quantas issues endereçadas (fix_version) e fechadas por release Issues com status 'Closed' e com fix_version direcionado a um versão específica - Não faz sentido - 

In [23]:
listIssuesFixVersion = []
for i in range(dfProjetosComCD.shape[0]):
    FVersionsDict = {}
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    tmp_name = dfProjetosComCD.iloc[i,0]
    for i in issues.query('issue_system_id == @tmp_issue_sistem & status == \'Closed\'').fix_versions:
        if isinstance(i, list): 
            for j in i:
                if j not in FVersionsDict:
                    FVersionsDict[j] = 1
                else:
                    FVersionsDict[j] += 1    
    for key in FVersionsDict:
        listIssuesFixVersion.append([tmp_name, tmp_issue_sistem, key, FVersionsDict[key] ])
df_Issues_fix_Version = pd.DataFrame(listIssuesFixVersion, columns=['name', 'issue_sistem_id', 'version', 'issues'])
df_Issues_fix_Version

,name,issue_sistem_id,version,issues
0,nifi,5bdafdb2f16c872096848fba,0.0.1,12
1,nifi,5bdafdb2f16c872096848fba,0.7.1,2
2,nifi,5bdafdb2f16c872096848fba,1.1.0,4
3,nifi,5bdafdb2f16c872096848fba,1.0.0,10
4,nifi,5bdafdb2f16c872096848fba,0.4.0,3
...,...,...,...,...
587,commons-jcs,5bef21d456f6a02953b578cf,jcs-1.2.7.9,4
588,commons-jcs,5bef21d456f6a02953b578cf,jcs-1.2.7.9.2,2
589,commons-jcs,5bef21d456f6a02953b578cf,jcs-2.0-beta-1,69
590,commons-jcs,5bef21d456f6a02953b578cf,jcs-1.3,13


Requer validação manual pra fazer isso, mas seria legal esses boxplots

In [24]:
tmp_issue_sistem = ObjectId('5bc052df762a157bad4494ca')
tmp_version = '3.6'
df_Issues_fix_Version.query('issue_sistem_id == @tmp_issue_sistem ').sort_values(by='version')

,name,issue_sistem_id,version,issues
360,commons-lang,5bc052df762a157bad4494ca,1.0.1,5
346,commons-lang,5bc052df762a157bad4494ca,2.0,70
345,commons-lang,5bc052df762a157bad4494ca,2.1,73
341,commons-lang,5bc052df762a157bad4494ca,2.2,56
344,commons-lang,5bc052df762a157bad4494ca,2.3,26
343,commons-lang,5bc052df762a157bad4494ca,2.4,49
338,commons-lang,5bc052df762a157bad4494ca,2.5,65
342,commons-lang,5bc052df762a157bad4494ca,2.6,4
340,commons-lang,5bc052df762a157bad4494ca,2.7,5
339,commons-lang,5bc052df762a157bad4494ca,3.0,180


In [25]:
#tmp_issue_sistem = ObjectId('5d7778c99f03373d37512b7e')
#tmp_version = '4.5.4'
df_Issues_fix_Version.query('issue_sistem_id == @tmp_issue_sistem & version < @tmp_version ').sort_values(by='version').issues.describe()

count     19.000000
mean      47.000000
std       48.188288
min        1.000000
25%        5.500000
50%       31.000000
75%       68.000000
max      180.000000
Name: issues, dtype: float64

In [26]:
df_Issues_fix_Version.query('issue_sistem_id == @tmp_issue_sistem & version >= @tmp_version ').sort_values(by='version').issues.describe()

count     8.000000
mean     12.375000
std      17.816024
min       1.000000
25%       1.750000
50%       3.000000
75%      15.500000
max      51.000000
Name: issues, dtype: float64

#### 3) Quantas issues fechadas por periodo (e prioridade) Issues com status 'Closed' e com updated_at entre os periodos das relases

Algoritmo Final:

In [27]:
averages_by_project = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    tmp_date = dfProjetosComCD.iloc[i,7]
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    ttm_tags_no_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date < @tmp_date ').sort_values(by='date')
    ttm_tags_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date >= @tmp_date ').sort_values(by='date')
    
    average_no_cd = 0
    for j in range(0,ttm_tags_no_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_no_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_no_cd.iloc[j+1,6]
        average_no_cd += issues.query('issue_system_id == @tmp_issue_sistem & status == \'Closed\' & updated_at > @data_release_anterior  & updated_at <= @data_release_da_vez').shape[0]
    average_no_cd = average_no_cd/(ttm_tags_no_cd.shape[0] - 1)
    
    average_cd = 0
    for j in range(0,ttm_tags_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_cd.iloc[j+1,6]
        average_cd += issues.query('issue_system_id == @tmp_issue_sistem & status == \'Closed\' & updated_at > @data_release_anterior  & updated_at <= @data_release_da_vez').shape[0]
    average_cd = average_cd/(ttm_tags_cd.shape[0] - 1)

    averages_by_project.append((tmp_name, tmp_vcs, average_no_cd, average_cd))

df_media_issues_periodo = pd.DataFrame(averages_by_project, columns=['name', 'vcs_system_id', 'no_CD', 'CD'])
    

In [28]:
issues.query('issue_system_id == @tmp_issue_sistem & status == \'Closed\' ').issue_type.value_counts()

Bug              105
Improvement       27
Question          17
New Feature        6
Wish               5
Documentation      2
Task               1
Name: issue_type, dtype: int64

In [29]:
print('Projetos com media de issues fechadas maior após CD:', df_media_issues_periodo.query('no_CD < CD').count().iloc[0] )
print('Projetos com media de issues fechadas menor após CD:', df_media_issues_periodo.query('no_CD > CD').count().iloc[0] )

Projetos com media de issues fechadas maior após CD: 7
Projetos com media de issues fechadas menor após CD: 14


#### 4) Bugs reportados por release (affects_versions) -- Mais precisa -- Issues do tipo 'Bug' com affects_versions direcionado a um versão específica

In [30]:
listIssuesAffectsVersions = []
for i in range(dfProjetosComCD.shape[0]):
    AfVersionsDict = {}
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    tmp_name = dfProjetosComCD.iloc[i,0]
    for i in issues.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\'').affects_versions:
        if isinstance(i, list): 
            for j in i:
                if j not in AfVersionsDict:
                    AfVersionsDict[j] = 1
                else:
                    AfVersionsDict[j] += 1    
    for key in AfVersionsDict:
        tmp_key = re.sub("[^\d\.]", "", key)
        listIssuesAffectsVersions.append([tmp_name, tmp_issue_sistem, tmp_key, AfVersionsDict[key] ])
df_Issues_Affects_versions = pd.DataFrame(listIssuesAffectsVersions, columns=['name', 'issue_sistem_id', 'version', 'bugs'])
df_Issues_Affects_versions

,name,issue_sistem_id,version,bugs
0,nifi,5bdafdb2f16c872096848fba,0.0.1,53
1,nifi,5bdafdb2f16c872096848fba,0.1.0,47
2,nifi,5bdafdb2f16c872096848fba,0.0.2,33
3,nifi,5bdafdb2f16c872096848fba,0.4.1,41
4,nifi,5bdafdb2f16c872096848fba,0.5.1,29
...,...,...,...,...
717,commons-jcs,5bef21d456f6a02953b578cf,2.01,24
718,commons-jcs,5bef21d456f6a02953b578cf,2.0,9
719,commons-jcs,5bef21d456f6a02953b578cf,2.02,7
720,commons-jcs,5bef21d456f6a02953b578cf,2.1,4


Apos validação manual da versão de adoção no issue sistem

In [31]:
adoption_version_issues_system = ['1.0.0', '1.5.4.1', '2.5','1.1.0','1.7.2','1.14','4.4.11','2.7.1', '4.2.0', '4.5.4', '3.6', '2.3.0', '0.4', '2.2', '0.53.0', '4.0.5', '0.9.0', '6.2', '1.8.2', '2.0.11', '2.1']
dfProjetosComCD['adoption_version_issues_system'] = adoption_version_issues_system

In [32]:
listmedia_no_cd = []
listmedia_cd = []
listmediana_no_cd = []
listmediana_cd = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    tmp_version = dfProjetosComCD.iloc[i,12]
    listmedia_no_cd.append(df_Issues_Affects_versions.query('issue_sistem_id == @tmp_issue_sistem & version < @tmp_version').bugs.mean())
    listmedia_cd.append(df_Issues_Affects_versions.query('issue_sistem_id == @tmp_issue_sistem & version >= @tmp_version').bugs.mean())
    listmediana_no_cd.append(df_Issues_Affects_versions.query('issue_sistem_id == @tmp_issue_sistem & version < @tmp_version').bugs.median())
    listmediana_cd.append(df_Issues_Affects_versions.query('issue_sistem_id == @tmp_issue_sistem & version >= @tmp_version').bugs.median())
dfProjetosComCD['afv_mean_open_bugs_no_cd'] = listmedia_no_cd
dfProjetosComCD['afv_mean_open_bugs_cd'] = listmedia_cd
dfProjetosComCD['afv_median_open_bugs_no_cd'] = listmediana_no_cd
dfProjetosComCD['afv_median_open_bugs_cd'] = listmediana_cd

In [34]:
print('Projetos com media de Bugs reportados explicitamente maior após CD:', dfProjetosComCD.query('afv_mean_open_bugs_no_cd < afv_mean_open_bugs_cd').count().iloc[0])
print('Projetos com media de Bugs reportados explicitamente menor após CD:', dfProjetosComCD.query('afv_mean_open_bugs_no_cd >= afv_mean_open_bugs_cd').count().iloc[0])
print('Projetos com mediana de Bugs reportados explicitamente maior após CD:', dfProjetosComCD.query('afv_median_open_bugs_no_cd < afv_median_open_bugs_cd').count().iloc[0] )
print('Projetos com mediana de Bugs reportados explicitamente menor após CD:', dfProjetosComCD.query('afv_median_open_bugs_no_cd >= afv_median_open_bugs_cd').count().iloc[0] )

Projetos com media de Bugs reportados explicitamente maior após CD: 3
Projetos com media de Bugs reportados explicitamente menor após CD: 18
Projetos com mediana de Bugs reportados explicitamente maior após CD: 4
Projetos com mediana de Bugs reportados explicitamente menor após CD: 17


#### 5) Bugs reportados (abertos) no periodo por release -- Mais factual -- Issues do tipo 'Bug' e com created_at entre os periodos das relases

In [35]:
bugs_averages_by_project = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]
    tmp_vcs = dfProjetosComCD.iloc[i,1]
    tmp_date = dfProjetosComCD.iloc[i,7]
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    ttm_tags_no_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date < @tmp_date ').sort_values(by='date')
    ttm_tags_cd = tagsDeRelase.query('vcs_system_id == @tmp_vcs & date >= @tmp_date ').sort_values(by='date')
    
    bugs_average_no_cd = 0
    bugs_median_no_cd = 0
    aux_array = np.zeros((ttm_tags_no_cd.shape[0] - 1))
    for j in range(0,ttm_tags_no_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_no_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_no_cd.iloc[j+1,6]
        aux_array[j] = (issues.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & created_at > @data_release_anterior  & created_at <= @data_release_da_vez').shape[0])
    bugs_average_no_cd = np.mean(aux_array)
    bugs_median_no_cd = np.median(aux_array)
    
    bugs_average_cd = 0
    bugs_median_cd = 0
    aux_array = np.zeros((ttm_tags_cd.shape[0] - 1))
    for j in range(0,ttm_tags_cd.shape[0] - 1):
        data_release_anterior = ttm_tags_cd.iloc[j,6]
        data_release_da_vez = ttm_tags_cd.iloc[j+1,6]
        aux_array[j] = (issues.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & created_at > @data_release_anterior  & created_at <= @data_release_da_vez').shape[0])
    bugs_average_cd = np.mean(aux_array)
    bugs_median_cd = np.median(aux_array)

    bugs_averages_by_project.append((tmp_name, tmp_vcs, bugs_average_no_cd, bugs_average_cd, bugs_median_no_cd, bugs_median_cd))
df_media_bugs_abertos_entre_releases = pd.DataFrame(bugs_averages_by_project, columns=['name', 'vcs_system_id', 'no_CD_mean', 'CD_mean', 'no_CD_median', 'CD_median'])
df_media_bugs_abertos_entre_releases

,name,vcs_system_id,no_CD_mean,CD_mean,no_CD_median,CD_median
0,nifi,5bd0268235e3ea2b7bbfdbae,68.000000,82.000000,49.5,90.0
1,kylin,5b43737430a71b06bcbdd168,43.043478,62.727273,19.0,47.0
2,struts,5b23df3c30a71b06bc01e16a,23.046512,6.882353,12.0,4.0
3,storm,5b7ffe9530a71b06bc70c038,71.142857,32.100000,51.5,10.5
4,opennlp,5b27c64830a71b06bceb0ead,24.500000,12.000000,24.5,7.0
5,commons-compress,5bed61cc35e3ea2b7b2a1805,6.407407,7.000000,6.0,7.0
6,httpcomponents-core,5d8ccfb33dfe5ddb8590d841,6.807692,19.000000,6.5,19.0
7,manifoldcf,5c3f0a7c621a9a77b35f3add,16.156250,11.714286,13.0,11.0
8,curator,608b9b33c26a57681ebf90b6,5.344828,10.333333,0.0,5.0
9,httpcomponents-client,5cd2806d504acf99a4cd728c,22.592593,16.250000,22.0,15.5


In [36]:
print('Projetos com media de Bugs reportados maior após CD:', df_media_bugs_abertos_entre_releases.query('no_CD_mean < CD_mean').count().iloc[0] )
print('Projetos com media de Bugs reportados menor após CD:', df_media_bugs_abertos_entre_releases.query('no_CD_mean >= CD_mean').count().iloc[0] )
print('Projetos com mediana de Bugs reportados maior após CD:', df_media_bugs_abertos_entre_releases.query('no_CD_median < CD_median').count().iloc[0] )
print('Projetos com mediana de Bugs reportados menor após CD:', df_media_bugs_abertos_entre_releases.query('no_CD_median >= CD_median').count().iloc[0] )

Projetos com media de Bugs reportados maior após CD: 7
Projetos com media de Bugs reportados menor após CD: 14
Projetos com mediana de Bugs reportados maior após CD: 10
Projetos com mediana de Bugs reportados menor após CD: 11
